# This is the code to generate prediction images with the ESPCN network

In [1]:
import tensorflow as tf

import os
import math
import numpy as np
from PIL import Image
import PIL
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from einops import rearrange, reduce
import sys
sys.path.insert(0, '/datadrive/AACS2023')
from fdf256dataset import FDF256Dataset

2023-04-26 08:21:58.347080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 08:21:59.194666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Could not load pyspng. Defaulting to pillow image backend.


In [2]:
dataset_path = '/datadrive/FDF/dataset/val'
dataset = FDF256Dataset(dirpath=dataset_path, load_keypoints=True, transform=None)

Dataset loaded from: /datadrive/FDF/dataset/val. Number of samples:6531


In [3]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=22,
                            prefetch_factor=2, persistent_workers=True, pin_memory=True)

In [4]:
sys.path.append('/datadrive/AACS2023/ESPCN')
import ESPCN
from ESPCN.ESPCN import run_model, load_model, get_lowres_image, upscale_image

In [5]:
model_path = "/datadrive/AACS2023/ESPCN+FDF256/ESPCN+FDF_model"

In [6]:
# Run the entire dataset through the network
from tqdm import tqdm
import torchvision.transforms as T
import torch
from einops import rearrange

crop_size = 256
upscale_factor = 4
input_size = crop_size // upscale_factor

images = []

for i in tqdm(range(len(dataset))):
    image = dataset[i]["img"]
    transform = T.ToPILImage()
    image = transform(image)
    images.append(image)
    

model = load_model(model_path=model_path)
model.trainable = False

for index, img in tqdm(enumerate(images)):
    lowres_input = get_lowres_image(img, upscale_factor)
    prediction = upscale_image(model, lowres_input)
    img_array = img_to_array(prediction)
    img_array = img_array.astype("float32") / 255.0
    plt.imsave(f'pred_ims_ESPCN+FDF/{index}.png', img_array)
    

100%|██████████| 6531/6531 [00:17<00:00, 376.44it/s]
2023-04-26 08:22:23.738821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14587 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0
2023-04-26 08:22:23.739811: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14587 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0002:00:00.0, compute capability: 7.0
2023-04-26 08:22:23.740545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14587 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0003:00:00.0, compute capability: 7.0
2023-04-26 08:22:23.741125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 14587 MB memory: 

1/1 [==============================] - 1s 1s/step


2023-04-26 08:22:25.660707: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
1it [00:01,  1.48s/it]

1/1 [==============================] - 0s 23ms/step


3it [00:01,  2.23it/s]

...

6528it [09:42, 11.77it/s]

1/1 [==============================] - 0s 22ms/step


6530it [09:42, 11.87it/s]

1/1 [==============================] - 0s 22ms/step


6531it [09:42, 11.21it/s]
